In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Transform

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

Dataset & DataLoader

In [3]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.42MB/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 444kB/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.07MB/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 5.15k/5.15k [00:00<?, ?B/s]


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



CNN Model

In [ ]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)  
        self.pool = nn.MaxPool2d(2, 2)              
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3) 
        
        self.fc1 = nn.Linear(64 * 5 * 5, 128)        
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  
        x = self.pool(self.relu(self.conv2(x)))  
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = FashionCNN()

Loss & Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training 

In [8]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0
    correct = 0
    total = 0
    
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}]  Loss: {running_loss:.3f}  Accuracy: {acc:.2f}%")


Epoch [1/10]  Loss: 449.540  Accuracy: 82.59%
Epoch [2/10]  Loss: 296.280  Accuracy: 88.48%
Epoch [3/10]  Loss: 253.048  Accuracy: 90.03%
Epoch [4/10]  Loss: 221.644  Accuracy: 91.28%
Epoch [5/10]  Loss: 201.222  Accuracy: 92.06%
Epoch [6/10]  Loss: 179.258  Accuracy: 92.95%
Epoch [7/10]  Loss: 160.793  Accuracy: 93.56%
Epoch [8/10]  Loss: 144.445  Accuracy: 94.16%
Epoch [9/10]  Loss: 129.097  Accuracy: 94.79%
Epoch [10/10]  Loss: 115.841  Accuracy: 95.39%


Evaluate

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"\nFinal Test Accuracy: {100 * correct / total:.2f}%")


Final Test Accuracy: 90.52%
